In [6]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

counts = np.array([v for v in word_dictionary.values()]).sum(axis=0)
print(counts)

[25748 67183]


Below, I have provided code skeletons. Your job is to make the code skeletons into an operational naive Bayes spam detector. (you may discard these skeletons if you would prefer to code this from scratch). Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:

In [7]:
#What is the prior P(Y=ham) ?
ham_prior = counts[1] / (counts[0] + counts[1])
spam_prior = counts[0] / (counts[0] + counts[1])

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
spam_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] =  val[1] / (val[0] + val[1])
    spam_likelihood[key] = val[0] / (val[0] + val[1])

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [11]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = ham_prior
    posterior_spam = spam_prior
    #! Don't forget to include the prior!
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler?
        try:
            posterior_ham *= ham_likelihood[w]
            posterior_spam *= spam_likelihood[w]
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, **make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.**

In [13]:
sucesses = 0
for i in range(len(test_labels)):
    test_label = test_labels[i]
    prediction = posteriors[i].argmax()
    if test_label == prediction:
        sucesses += 1
print(f'Percent correct = {(sucesses/len(test_labels))*100}')


Percent correct = 94.0754039497307
